In [1]:
import cv2
import av

def callback(frame):
    img = frame.to_ndarray(format="bgr24")

    img = cv2.cvtColor(cv2.Canny(img, 100, 200), cv2.COLOR_GRAY2BGR)
    
    # images.append(img)
    return av.VideoFrame.from_ndarray(img, format="bgr24")

cap=cv2.VideoCapture(0)
while True:
    # try to do cannay edge detection using opencv
    ret,frame=cap.read()
    frame=cv2.cvtColor(cv2.Canny(frame, 100, 200), cv2.COLOR_GRAY2BGR)
    cv2.imshow("frame",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
 


KeyboardInterrupt: 

: 

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

# Load the training data
X_train = []  # images of multiple people
y_train = []  # labels corresponding to the people in the images (e.g. 0, 1, 2, etc.)

# Load the pretrained PCA transformer
pca = PCA.load('path/to/pretrained_pca.pkl')

# Set up the classification pipeline
classifier = Pipeline([
    ('pca', pca),
    ('svm', SVC(kernel='linear', C=1.0))
])

# Train the classifier on the training data
classifier.fit(X_train, y_train)

# Now you can use the classifier to predict the label for a new image
new_image = []  # load a new image of one of the people
prediction = classifier.predict(new_image)
print(prediction)  # should output the label corresponding to the person in the image


In [3]:
# capture 10 images and store them in X_train

import cv2
import numpy as np
import time


# extract the face from the training image sample
def face_extractor(img):
    # load the face cascade
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    # convert to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    # if no face detected
    if faces == ():
        return None
    # crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]
    return cropped_face


 # initialize webcam
cap = cv2.VideoCapture(0)
count = 0
# collect 100 samples of your face from webcam input
while True:
    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (168,192))
        # time.sleep(0.)
        # save file in specified directory with unique name
        file_name_path = 'testfaces\\test_' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)
        # put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
    else:
        print("Face not found")
        pass
    if cv2.waitKey(1) == 13 or count == 1: #13 is the Enter Key
        break
    
cap.release()

cv2.destroyAllWindows()


C:\Users\Owner\AppData\Local\Temp\ipykernel_23788\1245499946.py:17: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if faces == ():


In [18]:
import os
from matplotlib import pyplot as plt
custom_images = []
input_faces = os.listdir('testfaces')
for face in input_faces:
    testFace = (plt.imread('testfaces/'+face))
    # preprocess faces to match the size of the training data
    testFace = np.mean(testFace,axis=2).T.flatten()
    custom_images.append(testFace)
    # print(testFace.shape)


# concatenate custom faces with original faces
custom_images = np.array(custom_images)
custom_images = custom_images.T
X_train = custom_images

In [43]:
tester_face = (plt.imread('test_1.jpg'))
tester_face = np.mean(tester_face,axis=2).T.flatten()
tester_face = tester_face.reshape(1,-1)
tester_face.shape[1]

32256

In [25]:
# X_train = X_train.T
X_train.shape[0]
# add 50 2's to the labels
for i in range(49):
    y_train = np.append(y_train, np.ones(1)*2)

In [45]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from joblib import dump, load
# Load the pretrained PCA transformer
pca = load('pca.pkl')
print("shape of X_train", X_train.shape[0], y_train.shape[0])
x_pca = pca.transform(X_train)
classf = SVC(kernel='rbf', class_weight='balanced', C=1e3, gamma=0.001)
classf.fit(x_pca, y_train)
# Now you can use the classifier to predict the label for a new image
new_image = []  # load a new image of one of the people
tester_face = pca.transform(tester_face)
prediction = classf.predict(tester_face)
print(prediction)  # should output the label corresponding to the person in the image


shape of X_train 100 100
[1.]


In [35]:
type(prediction)
int(prediction[0])

1

In [46]:
# a function to take in a face and return the prediction
def predict_face(face):
    face = np.mean(face,axis=2).T.flatten()
    face = face.reshape(1,-1)
    face = pca.transform(face)
    prediction = classf.predict(face)
    return int(prediction[0])

# a function to take in a whole image and return the prediction
def predict_image(image):
    # load the face cascade
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    # convert to grayscale
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    # detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    # if no face detected
    
    if faces is None:
        return None
    # crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = image[y:y+h, x:x+w]
    face = cv2.resize(cropped_face, (168,192))

    return predict_face(face)

predict_image(plt.imread('test_2.jpg'))

1